In [1]:
from src.fistanet.M5FISTANet import FISTANet
from src.fistanet.loader import DataSplit
from src.fistanet.solver import Solver
from os.path import join as pjoin
from torchsummary import summary
from datetime import datetime
import numpy as np
import torch

In [2]:
DATA_DIR = './data'
DATA_FILE_GEN = 'generated/BW_master_10000_2024-04-07-12-43-32.pkl'
DATA_FILE_SIGS = 'steinbrinker/testing_data_mvg_avg.npy'
DATA_FILE_BW = 'mit-bih/bw'
DATA_FILE_BPDN = 'generated/BW_alphas-BPDN_10000_2024-04-07-12-43-32.npy'
DICT_FILE_BW = 'steinbrinker/dictionary_BW_real_data.npy'
DATA_SIZE = 10000
BATCH_SIZE = 1000
TVT_SPLIT = {
    'train': 80,
    'valid': 10,
    'test': 10
}

FNET_LAYER_NO = 4
FNET_FEATURE_NO = 16
LAMBDA_SP_LOSS = 1e2
LAMBDA_SYM_LOSS = 1e-1

EPOCH_NO = 10000
TEST_EPOCH = 10001
LR_DEC_AFTER = 10000
LR_DEC_EVERY = 10
START_EPOCH = 2570
START_RUN = '2024-04-12-17-27-53'
LOG_INTERVAL = 4
LEARNING_RATE = 1e-3



# DATA_FILE_GEN = 'generated/BW_master_7999-8000_2024-04-07-12-43-32.pkl'
# DATA_SIZE = 2
# BATCH_SIZE = 1
# TVT_SPLIT = {
#     'train': 50,
#     'valid': 50,
#     'test': 0
# }
# FNET_LAYER_NO = 4
# FNET_FEATURE_NO = 16
# LEARNING_RATE = 1e-3
# LAMBDA_SP_LOSS = 0.5

In [3]:
trn_ldr, val_ldr, tst_ldr = DataSplit(DATA_DIR, DATA_FILE_GEN, DATA_FILE_SIGS, DATA_FILE_BW, DATA_FILE_BPDN, TVT_SPLIT, BATCH_SIZE)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
Psi = np.load(pjoin(DATA_DIR, DICT_FILE_BW))
Psi = torch.from_numpy(Psi)
Psi = Psi.clone().detach().to(device=device)

In [6]:
fista_net = FISTANet(FNET_LAYER_NO, FNET_FEATURE_NO)
fista_net = fista_net.to(device)# define arguments of fista_net


In [7]:
# summary(fista_net, input_size=(1, 64, 298), device=str(device))
print('Total number of parameters fista net:',
          sum(p.numel() for p in fista_net.parameters()))

Total number of parameters fista net: 18871


In [8]:
dt = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
if START_EPOCH:
    dt = START_RUN
args = {
    'model_name': 'FISTANet',
    'num_epochs': EPOCH_NO,
    'lr': LEARNING_RATE,
    'data_dir': DATA_DIR,
    'save_path': f'./runs/{dt}',
    'start_epoch': START_EPOCH,
    'start_run': START_RUN,
    'multi_gpu': False,
    'device': device,
    'log_interval': LOG_INTERVAL,
    'test_epoch': TEST_EPOCH,
    'lr_dec_after': LR_DEC_AFTER,
    'lr_dec_every': LR_DEC_EVERY,
    'lambda_sp_loss': LAMBDA_SP_LOSS,
    'lambda_sym_loss': LAMBDA_SYM_LOSS
}

In [9]:
solver = Solver(fista_net, Psi, trn_ldr, val_ldr, BATCH_SIZE, args, tst_ldr)

In [10]:
solver.train()

Training epoch 2571...

Train Epoch: 2571 [0/8000 (0%)]	Batch Loss: 183.382431	Learning Rate (w_theta): 0.001000	 TIME:6.0s
				Disc: 0.946941		Sym: 63.223553		Spars: 119.211937
	 TVw: 0.078003 | TVb: -2.067573 | GSw: -0.234357 | GSb: 0.065641 | TSUw: 0.465663 | TSUb: 0.034108

Train Epoch: 2571 [4000/8000 (50%)]	Batch Loss: 182.575513	Learning Rate (w_theta): 0.001000	 TIME:7.6s
				Disc: 0.906949		Sym: 62.470123		Spars: 119.198441
	 TVw: 0.077748 | TVb: -2.067631 | GSw: -0.234357 | GSb: 0.065641 | TSUw: 0.465663 | TSUb: 0.034108
Validating epoch 2571...
-------------------------------------------
Epoch statistics:
Average training loss: 174.87598375531007
Average validation loss: 25.373139509061453
Training epoch 2572...

Train Epoch: 2572 [0/8000 (0%)]	Batch Loss: 172.296090	Learning Rate (w_theta): 0.001000	 TIME:9.9s
				Disc: 0.982198		Sym: 58.437153		Spars: 112.876740
	 TVw: 0.077738 | TVb: -2.067682 | GSw: -0.234357 | GSb: 0.065641 | TSUw: 0.465663 | TSUb: 0.034108

Train Epoch

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 20000000 bytes.